In [1]:
PRECISION = 163
F = GF(2^163, 'g', modulus = x^163 + x^8 + x^2 + x + 1)
F.inject_variables()
R.<x,y> = F[]
exp = 2^PRECISION-2

Defining g


In [2]:
def str_to_poly(str):
	I=Integer(str, base=16)
	v=F(0)
	for i in range (0,F.degree()):
		if(I >> i) & 1 > 0:
			v = v + g^i
	return v

def poly_to_str(poly):
	vec=poly._vector_()
	string = ""
	for i in range(0,len(vec)):
		string = string + str(vec[len(vec) - i - 1])
	return hex(Integer(string, base=2))

In [3]:
rpoly = str_to_poly("0107")
r = poly_to_str(rpoly)
rinvpoly = rpoly^-1
rinv = poly_to_str(rinvpoly)
r2poly = rpoly*rpoly
r2 = poly_to_str(r2poly)

n = 0x080000000000000000000000000000000000000107

a = 0x072546b5435234a422e0789675f432c89435de5242

apoly = str_to_poly("072546b5435234a422e0789675f432c89435de5242")
#mapoly = apoly*r2poly*rinvpoly

b = 0x00c9517d06d5240d3cff38c74b20b6cd4d6f9dd4d9

bpoly = str_to_poly("00c9517d06d5240d3cff38c74b20b6cd4d6f9dd4d9")
#mbpoly = bpoly*r2poly*rinvpoly

E = EllipticCurve(y^2 + x*y - x^3 - apoly*x^2 - bpoly)

G = E((str_to_poly("07af69989546103d79329fcc3d74880f33bbe803cb"), str_to_poly("01ec23211b5966adea1d3f87f7ea5848aef0b7ca9f")))


In [4]:
def mont(*args):
    res = []
    for n in args:
        tres = n*rpoly
        res.append(tres)
    return res

In [5]:
def de_mont(*args):
    res = []
    for n in args:
        tres = n*rinvpoly
        res.append(tres)
    
    return res

In [6]:
apoly_mont = mont(apoly)[0]
bpoly_mont = mont(bpoly)[0]
#assert apoly_mont == apoly*rpoly

In [7]:
def aff_to_jac(r_x,r_y):
    return r_x,r_y,rpoly


In [8]:
def jac_to_aff(r_x,r_y,r_z):
    r_z_inv = (r_z^exp)*rpoly #works somehow
    #r_z_inv = r_z^-1
    #r_z_inv = r_z^exp*rpoly
    r_z2_inv = r_z_inv*r_z_inv
    r_z3_inv = r_z2_inv*r_z_inv
    
    r_x_a = r_x*r_z2_inv
    r_y_a = r_y*r_z3_inv
    
    return r_x_a,r_y_a

In [9]:
def point_add_jac_mont(X1,Y1,Z1,X2,Y2,Z2):
    
    O1 = Z1*Z1*rinvpoly
    O2 = Z2*Z2*rinvpoly

    A = X1*O2*rinvpoly
    B = X2*O1*rinvpoly

    T1 = Y1*O2*rinvpoly
    C = T1*Z2*rinvpoly

    T2 = Y2*O1*rinvpoly
    D = T2*Z1*rinvpoly

    E = A+B

    F = C+D

    G = E*Z1*rinvpoly

    T3 = F*X2*rinvpoly
    T4 = G*Y2*rinvpoly
    H = T3+T4

    Z3 = G*Z2*rinvpoly

    I = F+Z3

    T5 = Z3*Z3*rinvpoly
    T6 = apoly_mont*T5*rinvpoly
    T7 = F*I*rinvpoly
    T8 = T6+T7
    T9 = E*E*rinvpoly
    T10 = T9*E*rinvpoly
    X3 = T8+T10

    T11 = I*X3*rinvpoly
    T12 = G*G*rinvpoly
    T13 = T12*H*rinvpoly
    Y3 = T11+T13
    
    return X3,Y3,Z3

In [10]:
def point_double_jac_mont(X1,Y1,Z1):
    
    A = X1*X1*rinvpoly
    B = A*A*rinvpoly
    C = Z1*Z1*rinvpoly
    D = C*C*rinvpoly
    
    T1 = D*D*rinvpoly
    T2 = bpoly_mont*T1*rinvpoly
    X3 = B+T2
    
    Z3 = X1*C*rinvpoly
    
    T3 = B*Z3*rinvpoly
    T4 = Y1*Z1*rinvpoly
    T5 = A+T4
    T6 = T5+Z3
    T7 = T6*X3*rinvpoly
    Y3 = T3+T7

    return X3,Y3,Z3

In [11]:
def point_mult_jac_mont(X1,Y1,Z1,k):
    R_X = X1
    R_Y = Y1
    R_Z = Z1

    k_bin = bin(k)[3:]
    #print(bin(k),"\n",k_bin)
    for x in k_bin:
        R_X,R_Y,R_Z = point_double_jac_mont(R_X,R_Y,R_Z)#j
        if(x=="1"):
            R_X,R_Y,R_Z = point_add_jac_mont(R_X,R_Y,R_Z,X1,Y1,Z1)#j
            
    return R_X,R_Y,R_Z

## Testing Point addition

In [12]:
P = E((str_to_poly("5973f2ce9481d19643a811a897599e6752b62834"), str_to_poly("655ebc626a619726018b03f84730e81a619142c94")))
Q = E((str_to_poly("525bd3cdda7c989f21652da0de9c7a5d35ae62f1f"), str_to_poly("54972dfe42d888584d8cfce0757f7b0a64801ae68")))


##### Montgomeraizing points

In [13]:
k = Integer(getrandbits(PRECISION-1))
P = k*G
#print(str_to_poly(P))

In [14]:
P_MD = mont(P.xy()[0],P.xy()[1],Q.xy()[0],Q.xy()[1])

In [15]:
assert P_MD[3] == Q.xy()[1]*rpoly

##### Affine to jacobi

In [16]:
P_MD_jac_x, P_MD_jac_y, P_MD_jac_z = aff_to_jac(P_MD[0],P_MD[1])

Q_MD_jac_x, Q_MD_jac_y, Q_MD_jac_z = aff_to_jac(P_MD[2],P_MD[3])

##### Addition

In [17]:
R_MD_jac_x, R_MD_jac_y, R_MD_jac_z = point_add_jac_mont(P_MD_jac_x, P_MD_jac_y, P_MD_jac_z, Q_MD_jac_x, Q_MD_jac_y, Q_MD_jac_z)

##### Jacobi to Affine

In [18]:
R_MD_x, R_MD_y = jac_to_aff(R_MD_jac_x, R_MD_jac_y, R_MD_jac_z)

##### Demontgomerizing Points

In [19]:
R = de_mont(R_MD_x,R_MD_y)

##### Result and varification

In [20]:
P_add = P+Q
print(poly_to_str(P_add.xy()[0]),poly_to_str(P_add.xy()[1]))
print(poly_to_str(R[0]),poly_to_str(R[1]))

0xb8be650aab48695aa74fd5f2ed7b5967b05c543a 0x5e848ca7aa4521d5eb6af03f0b42c8584af0198b5
0xb8be650aab48695aa74fd5f2ed7b5967b05c543a 0x5e848ca7aa4521d5eb6af03f0b42c8584af0198b5


In [21]:
#Demont then jac - affine

Ra = de_mont(R_MD_jac_x, R_MD_jac_y, R_MD_jac_z)


In [22]:
Ra_x, Ra_y = jac_to_aff(Ra[0], Ra[1], Ra[2])

In [23]:
print(poly_to_str(Ra_x),poly_to_str(Ra_y))

0x7b4ab0ac2dbb5fed59b7f306cd6441ade20c53410 0x38b31c9746730502508fd80cbb95be36974287f1e


## Testing Point Doubling

In [24]:
D = P

##### Montgomerizing

In [25]:
D_MD = mont(D.xy()[0],D.xy()[1])
assert D_MD[1] == D.xy()[1]*rpoly

##### Affine to jacobi

In [26]:
D_MD_jac_x, D_MD_jac_y, D_MD_jac_z = aff_to_jac(D_MD[0],D_MD[1])

##### Doubling

In [27]:
R_D_MD_jac_x, R_D_MD_jac_y, R_D_MD_jac_z = point_double_jac_mont(D_MD_jac_x, D_MD_jac_y, D_MD_jac_z)

##### Jacobi to Affine

In [28]:
R_D_MD_x, R_D_MD_y = jac_to_aff(R_D_MD_jac_x, R_D_MD_jac_y, R_D_MD_jac_z)

##### Demontgomerizing Points

In [29]:
R_D = de_mont(R_D_MD_x, R_D_MD_y)

##### Result and varification

In [30]:
D_dub = 2*D
print(poly_to_str(D_dub.xy()[0]),poly_to_str(D_dub.xy()[1]))
print(poly_to_str(R_D[0]),poly_to_str(R_D[1]))

0x3039a192ad4eb55fcaf35e4ade3f8564277435278 0x4a70749b13a6634a6b206efa7f308da32a1f81d46
0x3039a192ad4eb55fcaf35e4ade3f8564277435278 0x4a70749b13a6634a6b206efa7f308da32a1f81d46


## Testing Point Multiplication

In [31]:
M = P

In [32]:
I = Integer(getrandbits(PRECISION-1)); print(I)

3630307443970571539501475810076920358293737507748


##### Montgomerizing

In [33]:
M_MD = mont(M.xy()[0],M.xy()[1])
assert M_MD[1] == M.xy()[1]*rpoly

##### Affine to jacobi

In [34]:
M_MD_jac_x, M_MD_jac_y, M_MD_jac_z = aff_to_jac(M_MD[0],M_MD[1])

##### Doubling

In [35]:
R_M_MD_jac_x, R_M_MD_jac_y, R_M_MD_jac_z = point_mult_jac_mont(M_MD_jac_x, M_MD_jac_y, M_MD_jac_z,I)

##### Jacobi to Affine

In [36]:
R_M_MD_x, R_M_MD_y = jac_to_aff(R_M_MD_jac_x, R_M_MD_jac_y, R_M_MD_jac_z)

##### Demontgomerizing Points

In [37]:
R_M = de_mont(R_M_MD_x, R_M_MD_y)

##### Result and varification

In [38]:
M_Mul = I*M
print(poly_to_str(M_Mul.xy()[0]),poly_to_str(M_Mul.xy()[1]))
print(poly_to_str(R_M[0]),poly_to_str(R_M[1]))

0x15b73551d750286804b7e6296a96ecf6793b5ac74 0xb89210e8ef1211ffd92197417518af440db9ba77
0x15b73551d750286804b7e6296a96ecf6793b5ac74 0xb89210e8ef1211ffd92197417518af440db9ba77
